# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [2]:
!pip install gdown==3.6.0

  Running setup.py bdist_wheel for gdown ... done
  Stored in directory: /root/.cache/pip/wheels/97/90/fa/25654eb65da3e6da7752db71a164e0eb8f7a6fb4335eeb46ab
Successfully built gdown
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
my_file_id = "1xIgZQknlp3IIa5jsgBoINp9rpF0JiUGe" # https://drive.google.com/file/d/1xIgZQknlp3IIa5jsgBoINp9rpF0JiUGe/view?usp=sharing
!gdown https://drive.google.com/uc?id={my_file_id}

Downloading...
From: https://drive.google.com/uc?id=1xIgZQknlp3IIa5jsgBoINp9rpF0JiUGe
To: /home/workspace/model_checkpoint_B_2019-01-09 15:10:53.pt
234MB [00:01, 145MB/s]  


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

# Classifier class coded to work like the example above.
class FFClassifier(nn.Module):
    def __init__(self, in_features, hidden_1, hidden_2, 
                 out_features, drop_prob=0.5):
        super().__init__()
        self.fc1 = nn.Linear(in_features, out_features)
        
        #self.fc1 = nn.Linear(in_features, hidden_1)
        #self.fc2 = nn.Linear(hidden_1, hidden_2)
        #self.fc3 = nn.Linear(hidden_2, out_features)
        #self.drop = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.fc1(x)
        
        #x = self.drop(F.relu(self.fc1(x)))
        #x = self.drop(F.relu(self.fc2(x)))
        #x = self.fc3(x)
        
        x = F.log_softmax(x, dim=1)
        return x



def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location='cpu')

    model = models.resnet152(pretrained=False)
    
    for param in model.parameters():
       param.requires_grad = False

    # Put the classifier on the pretrained network
    model.fc = FFClassifier(2048, 
                                    checkpoint['hidden_1'], 
                                    checkpoint['hidden_2'], 
                                    102, 
                                    drop_prob=0.2)

    model.load_state_dict(checkpoint['state_dict'], strict=False)

    model.class_to_idx = checkpoint["class_to_idx"]
    model.idx_to_class = checkpoint["idx_to_class"]
    model.cat_to_name = checkpoint["cat_to_name"]
    model.hidden_1 = checkpoint["hidden_1"]
    model.hidden_2 = checkpoint["hidden_2"]

    model.eval()
    
    return model


# Load the model
model = load_checkpoint('/home/workspace/model_checkpoint_B_2019-01-09 15:10:53.pt')
#model = load_checkpoint('/home/workspace/model_checkpoint_C_2019-01-09 15:20:57.pt')
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224

# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.

# norm_mean = [0.485, 0.456, 0.406]
# norm_std = [0.229, 0.224, 0.225]
norm_mean = [0.5178361839861569, 0.4106749456881299, 0.32864167836880803]
norm_std = [0.2972239085211309, 0.24976049135203868, 0.28533308036347665]

In [2]:
# https://github.com/GabrielePicco/deep-learning-flower-identifier

!git clone https://github.com/GabrielePicco/deep-learning-flower-identifier
!pip install requests
!pip install airtable

import sys
sys.path.insert(0, 'deep-learning-flower-identifier')
from test_model_pytorch_facebook_challenge import calc_accuracy

# model = load_your_model('classifier.pth')
calc_accuracy(model, input_image_size=224, use_google_testset=False, norm_mean=norm_mean, norm_std=norm_std)

fatal: destination path 'deep-learning-flower-identifier' already exists and is not an empty directory.
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Running setup.py bdist_wheel for airtable ... done
  Stored in directory: /root/.cache/pip/wheels/9b/ba/63/364c02fabcd50ef6e2f101a57feb727bd7a693697765a9df17
Successfully built airtable
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 1.0
Batch accuracy (Size 32): 0.96875
Batch accuracy (Size 32): 1.0
Batch accuracy (Size

0.99399036

In [3]:
# Data option 2 - Download train + validation + testing data

!wget -cq https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz
!rm -rf flower_data
!mkdir flower_data
!tar -xzf "flower_data.tar.gz" --directory flower_data

# Correct misslabelled valid data.
!mv flower_data/valid/93/image_07303.jpg flower_data/valid/94
!mv flower_data/valid/96/image_07677.jpg flower_data/valid/97

# Correct misslabelled test data.
!mv flower_data/test/96/image_07683.jpg flower_data/test/97
!mv flower_data/test/96/image_07676.jpg flower_data/test/97

In [4]:
# Load some data for testing the loaded model.

import time
import math
from IPython.display import HTML, display

from torchvision import datasets, transforms, models

data_dir = 'flower_data'
#final_test_dir = data_dir + '/valid'
final_test_dir = data_dir + '/test'

data_transforms = {'final_test': transforms.Compose([transforms.Resize(256, interpolation=2),
                                                #===
                                                transforms.CenterCrop(224),
                                                #===
                                                # transforms.Grayscale(num_output_channels=3),
                                                transforms.ToTensor(),
                                                transforms.Normalize(norm_mean, 
                                                                     norm_std),
                                               ])
                  }

# Load the datasets with ImageFolder
image_datasets = {'final_test': datasets.ImageFolder(final_test_dir, transform=data_transforms['final_test'])}

# Using the image datasets and the trainforms, define the dataloaders
batch_size = 4

dataloaders = {'final_test': torch.utils.data.DataLoader(image_datasets['final_test'], batch_size=batch_size, shuffle=True)}


In [ ]:
# Test the loaded model.

def progress_bar(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

criterion = nn.NLLLoss()

test_loss_sum = 0.0
test_correct_count = 0.0

######################    
# test the model #
######################
test_start_time = time.time()
print("Testing...")
test_display = display(progress_bar(0, 100), display_id=True)

model.eval()    
num_batches = math.ceil(len(dataloaders['final_test'].dataset) / batch_size)
batch_stride = 1

for batch_idx, (images, labels) in enumerate(dataloaders['final_test']):
    if batch_idx % batch_stride == 0:
        with torch.set_grad_enabled(False):
            outputs = model(images)  # batch_size x 102
            loss = criterion(outputs, labels)  # Average loss value over batch# .

        test_loss_sum += loss.item() * images.size(0)

        _, predicted_labels = torch.max(outputs, -1)
        test_correct_count += (predicted_labels == labels).double().sum().item()

        progress = (batch_idx+1) * 100.0 / num_batches
        test_display.update(progress_bar(progress, 100))


test_end_time = time.time()

############################
# calculate average losses #
############################
test_loss = test_loss_sum * batch_stride / len(dataloaders['final_test'].dataset)
test_acc = test_correct_count * batch_stride / len(dataloaders['final_test'].dataset)

print('Testing Loss={:.6f}  Testing Accuracy={:.6f}  Duration={:.2f}'.format(test_loss, 
                                                                             test_acc, 
                                                                             test_end_time - test_start_time))



Testing...
